In [3]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

#import os
#for dirname, _, filenames in os.walk('/kaggle/input'):
#    for filename in filenames:
#        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
## You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [4]:
# import the necessay libraries
import zipfile
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.preprocessing import image
import imageio
import urllib

In [5]:
# setting up directory 
#file_name = '/kaggle/input/horses-or-humans-dataset'
training = '../input/horses-or-humans-dataset/horse-or-human/train/'

In [6]:
# unzipping the zip folder
#zip_ref = zipfile.ZipFile(file_name, 'r')
#zip_ref.extractall(training)
#zip_ref.close()

In [7]:
# load the data using ImageDataGenerator
# all images will be rescaled by 1.0/255
train_datagen = ImageDataGenerator(rescale = 1/255)

train_generator = train_datagen.flow_from_directory(training,
                                                   target_size = (300, 300),
                                                   class_mode = 'binary')

Found 1027 images belonging to 2 classes.


There are several major differences between this dataset and the Fashion MNIST one
that you have to take into account when designing an architecture for classifying the
images. First, the images are much larger—300 × 300 pixels—so more layers may be
needed. Second, the images are full color, not grayscale, so each image will have three
channels instead of one. Third, there are only two image types, so we have a binary
classifier that can be implemented using just a single output neuron, where it
approaches 0 for one class and 1 for the other. Keep these considerations in mind
when exploring this archit

There are a number of things to note here. First of all, this is the very first layer. We’re
defining 16 filters, each 3 × 3, but the input shape of the image is (300, 300, 3).
Remember that this is because our input image is 300 × 300 and it’s in color, so there
are three channels, instead of just one for the monochrome Fashion MNIST dataset
we were using earlierecture:

In [8]:
# build the model
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(16, (3,3), activation ='relu',
                          input_shape = (300, 300, 3)),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Conv2D(32, (3,3), activation ='relu'),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Conv2D(64, (3,3), activation ='relu'),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Conv2D(64, (3,3), activation ='relu'),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Conv2D(64, (3,3), activation ='relu'),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(512, activation = 'relu'),
    tf.keras.layers.Dense(1, activation = 'sigmoid')
])

/opt/conda/lib/python3.10/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [9]:
# model summary
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 298, 298, 16)   │           448 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 149, 149, 16)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 147, 147, 32)   │         4,640 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 73, 73, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 71, 71, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 35, 35, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_3 (Conv2D)               │ (None, 33, 33, 64)     │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_3 (MaxPooling2D)  │ (None, 16, 16, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_4 (Conv2D)               │ (None, 14, 14, 64)     │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_4 (MaxPooling2D)  │ (None, 7, 7, 64)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 3136)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 512)            │     1,606,144 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │           513 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,704,097 (6.50 MB)

 Trainable params: 1,704,097 (6.50 MB)

 Non-trainable params: 0 (0.00 B)

To train the network, we’ll have to compile it with a loss function and an optimizer. In
this case the loss function can be binary cross entropy loss function binary cross
entropy, because there are only two classes, and as the name suggests this is a loss
function that is designed for that scenario. And we can try a new optimizer, root mean
square propagation (RMSprop), that takes a learning rate (lr) parameter that allows us
to tweak the learning.

In [10]:
# compile the CNN network
model.compile(loss = 'binary_crossentropy',
             optimizer = tf.keras.optimizers.RMSprop(learning_rate = 0.001),
             metrics = ['accuracy'])

Over just 15 epochs, this architecture gives us a very impressive 100% accuracy on
the training set. Of course, this is just with the training data, and isn’t a  indication o 
performance on data that the network hasn’t previously see This performance is an indication tha the model has an overfitting on the train data.. we will need to eavluate it on the test data to see it how it performs on unseen datan.

In [14]:
# train on the train generator
history = model.fit(
    train_generator,
    epochs = 15
)

Epoch 1/15


/opt/conda/lib/python3.10/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


33/33 ━━━━━━━━━━━━━━━━━━━━ 59s 2s/step - accuracy: 0.5717 - loss: 0.6980
Epoch 2/15
33/33 ━━━━━━━━━━━━━━━━━━━━ 57s 2s/step - accuracy: 0.9042 - loss: 0.3277
Epoch 3/15
33/33 ━━━━━━━━━━━━━━━━━━━━ 56s 2s/step - accuracy: 0.9393 - loss: 0.2321
Epoch 4/15
33/33 ━━━━━━━━━━━━━━━━━━━━ 57s 2s/step - accuracy: 0.9656 - loss: 0.1072
Epoch 5/15
33/33 ━━━━━━━━━━━━━━━━━━━━ 57s 2s/step - accuracy: 0.9195 - loss: 0.6513
Epoch 6/15
33/33 ━━━━━━━━━━━━━━━━━━━━ 57s 2s/step - accuracy: 0.9891 - loss: 0.0593
Epoch 7/15
33/33 ━━━━━━━━━━━━━━━━━━━━ 82s 2s/step - accuracy: 0.9951 - loss: 0.0178
Epoch 8/15
33/33 ━━━━━━━━━━━━━━━━━━━━ 57s 2s/step - accuracy: 0.9845 - loss: 0.0699
Epoch 9/15
33/33 ━━━━━━━━━━━━━━━━━━━━ 56s 2s/step - accuracy: 0.9401 - loss: 0.2343
Epoch 10/15
33/33 ━━━━━━━━━━━━━━━━━━━━ 83s 2s/step - accuracy: 0.9955 - loss: 0.0274
Epoch 11/15
33/33 ━━━━━━━━━━━━━━━━━━━━ 57s 2s/step - accuracy: 0.9901 - loss: 0.0432
Epoch 12/15
33/33 ━━━━━━━━━━━━━━━━━━━━ 57s 2s/step - accuracy: 0.9955 - loss: 0.0197


## Adding Validation to the Horses or Humans Dataset

In [11]:
# get the validation data
validation_dir = '../input/horses-or-humans-dataset/horse-or-human/validation/'

# set up the image generator for validation
validation_datagen = ImageDataGenerator(rescale= 1/255)
validation_gen = validation_datagen.flow_from_directory(
    validation_dir,
    target_size = (300, 300),
    class_mode = 'binary'
)

Found 256 images belonging to 2 classes.


To have TensorFlow perform the validation for you, you simply update your 
model.fir method to indicate that you want to use the validation data t 
test the model epoch by epoch. You do this by using the validation_data paramet r
and passing it the validation generator you just construct

After training for 15 epochs, you should see that your model is 100%+ accurate on the
training set, but only about 88% on the validation set. This is an indication that the
model is overfitting, as we saw in the previous chapter.ed:

In [12]:
# validate the model on the validation set
history = model.fit(
    train_generator,
    epochs = 15,
    validation_data = validation_gen
)

Epoch 1/15


/opt/conda/lib/python3.10/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


33/33 ━━━━━━━━━━━━━━━━━━━━ 65s 2s/step - accuracy: 0.5899 - loss: 0.6943 - val_accuracy: 0.8281 - val_loss: 0.3740
Epoch 2/15
33/33 ━━━━━━━━━━━━━━━━━━━━ 60s 2s/step - accuracy: 0.8195 - loss: 0.4499 - val_accuracy: 0.8320 - val_loss: 0.9249
Epoch 3/15
33/33 ━━━━━━━━━━━━━━━━━━━━ 82s 2s/step - accuracy: 0.8934 - loss: 0.3929 - val_accuracy: 0.7930 - val_loss: 0.7396
Epoch 4/15
33/33 ━━━━━━━━━━━━━━━━━━━━ 60s 2s/step - accuracy: 0.9548 - loss: 0.1807 - val_accuracy: 0.8945 - val_loss: 0.6588
Epoch 5/15
33/33 ━━━━━━━━━━━━━━━━━━━━ 60s 2s/step - accuracy: 0.9663 - loss: 0.1115 - val_accuracy: 0.8984 - val_loss: 0.7605
Epoch 6/15
33/33 ━━━━━━━━━━━━━━━━━━━━ 81s 2s/step - accuracy: 0.9830 - loss: 0.0482 - val_accuracy: 0.8789 - val_loss: 1.1052
Epoch 7/15
33/33 ━━━━━━━━━━━━━━━━━━━━ 61s 2s/step - accuracy: 0.9641 - loss: 0.1315 - val_accuracy: 0.8633 - val_loss: 1.1555
Epoch 8/15
33/33 ━━━━━━━━━━━━━━━━━━━━ 61s 2s/step - accuracy: 0.9796 - loss: 0.2104 - val_accuracy: 0.8945 - val_loss: 1.0239
Epo

## Testing the Model

There are a few important points to consider here. First, even though the network
was trained on synthetic, computer-generated imagery, itstill  performs quite well at spo
ting horsebut is not able to identifying humans in real photographs.s. This is a potential boon in that you m or may y
not need thousands of photographs to train a model, and can do it relatively chea ly
with  But this dataset also demonstrates a fundamental issue you will face. Your training set
cannot hope to represent every possible scenario your model might face in the wild 
and thus the model will always have some level of overspecializati n toward the trn‐
ing setCGI.

In [43]:
import os
import numpy as np
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import load_model

# Define the path to the folder containing your images and the model path
image_folder_path = '../input/picture-model/'


# Set the image size to match the input size of the model
img_size = (300, 300, 3)  # Change this based on your model's input size

# Loop through each image in the folder
for img_name in os.listdir(image_folder_path):
    img_path = os.path.join(image_folder_path, img_name)
    
    # Load and preprocess the image
    img = image.load_img(img_path, target_size=img_size)
    img_array = image.img_to_array(img) / 255.0  # Normalize if required by your model
    img_array = np.expand_dims(img_array, axis=0)  # Add batch dimension

    # Predict the class
    prediction = model.predict(img_array)
    
    # Get the prediction result
    if classes[0]>0.5:
         print(img_name + " is a human")
    else:
         print(img_name + " is a horse")
    
    #print(f"Image: {img_name} - Prediction: {class_label}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
bird.jpg is a horse
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
pic_two.jpg is a horse
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
horse_two.jpg is a horse
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
horse_one.jpg is a horse
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
pic_three.jpg is a horse
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
pic_one.jpg is a horse
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
horse_three.jpg is a horse


## Image Augmentation

In the previous section, you built a horse-or-human classifier model that was trained 
on a relatively small dataset. As a result, you soon began to hit problems classifyin 
some previously unseen images, such as t e miscategorization of humanan with a
horse because the training set didn’t include any images of people in that pose

In [45]:
# add extra transformation to the image generator
train_datagen = ImageDataGenerator(rescale = 1/255,
                                  rotation_range = 40,
                                  width_shift_range = 0.2,
                                  height_shift_range = 0.2,
                                  shear_range = 0.2,
                                  horizontal_flip = True,
                                  fill_mode = 'nearest')

train_generator = train_datagen.flow_from_directory(training,
                                                   target_size = (300, 300),
                                                   class_mode = 'binary')

Found 1027 images belonging to 2 classes.


In [46]:
# retrain the train image and compare to validation set
history = model.fit(
    train_generator,
    epochs = 15,
    validation_data = validation_gen
)

Epoch 1/15


/opt/conda/lib/python3.10/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


33/33 ━━━━━━━━━━━━━━━━━━━━ 73s 2s/step - accuracy: 0.7798 - loss: 0.7409 - val_accuracy: 0.8281 - val_loss: 1.0970
Epoch 2/15
33/33 ━━━━━━━━━━━━━━━━━━━━ 73s 2s/step - accuracy: 0.8605 - loss: 0.3377 - val_accuracy: 0.8984 - val_loss: 0.7509
Epoch 3/15
33/33 ━━━━━━━━━━━━━━━━━━━━ 72s 2s/step - accuracy: 0.8854 - loss: 0.3319 - val_accuracy: 0.8633 - val_loss: 1.1241
Epoch 4/15
33/33 ━━━━━━━━━━━━━━━━━━━━ 72s 2s/step - accuracy: 0.8978 - loss: 0.2495 - val_accuracy: 0.8945 - val_loss: 0.5514
Epoch 5/15
33/33 ━━━━━━━━━━━━━━━━━━━━ 83s 2s/step - accuracy: 0.9356 - loss: 0.1834 - val_accuracy: 0.8086 - val_loss: 2.0778
Epoch 6/15
33/33 ━━━━━━━━━━━━━━━━━━━━ 73s 2s/step - accuracy: 0.9179 - loss: 0.2028 - val_accuracy: 0.8320 - val_loss: 1.0879
Epoch 7/15
33/33 ━━━━━━━━━━━━━━━━━━━━ 73s 2s/step - accuracy: 0.9542 - loss: 0.1278 - val_accuracy: 0.7266 - val_loss: 3.3469
Epoch 8/15
33/33 ━━━━━━━━━━━━━━━━━━━━ 72s 2s/step - accuracy: 0.9442 - loss: 0.1605 - val_accuracy: 0.8320 - val_loss: 1.2057
Epo

In [47]:
import os
import numpy as np
from tensorflow.keras.preprocessing import image
#from tensorflow.keras.models import load_model

# Define the path to the folder containing your images and the model path
image_folder_path = '../input/picture-model/'


# Set the image size to match the input size of the model
img_size = (300, 300, 3)  # Change this based on your model's input size

# Loop through each image in the folder
for img_name in os.listdir(image_folder_path):
    img_path = os.path.join(image_folder_path, img_name)
    
    # Load and preprocess the image
    img = image.load_img(img_path, target_size=img_size)
    img_array = image.img_to_array(img) / 255.0  # Normalize if required by your model
    img_array = np.expand_dims(img_array, axis=0)  # Add batch dimension

    # Predict the class
    prediction = model.predict(img_array)
    
    # Get the prediction result
    if classes[0]>0.5:
         print(img_name + " is a human")
    else:
         print(img_name + " is a horse")
    
    #print(f"Image: {img_name} - Prediction: {class_label}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
bird.jpg is a horse
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
pic_two.jpg is a horse
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
horse_two.jpg is a horse
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
horse_one.jpg is a horse
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
pic_three.jpg is a horse
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
pic_one.jpg is a horse
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
horse_three.jpg is a horse


## Transfer Learning

In [3]:
from tensorflow.keras.applications.inception_v3 import InceptionV3

In [11]:
import h5py

# get the weights of the model url
weights_url = "../input/inception-v3/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5"

weights_file = h5py.File(weights_url, "r")
weights_file

<HDF5 file "inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5" (mode r)>